### Opening a ROOT file

#### Data sample

In [ ]:
import ROOT
import os
files_path = '../files/'
file_name = os.path.join(files_path, 'preload_dataReader_2016_bin3.root')
data_file = ROOT.TFile(file_name)

#### Signal sample

In [ ]:
sig_file_name = os.path.join(files_path, 'preload_sigMCReader_2016_bin3.root')
sig_file = ROOT.TFile(sig_file_name)

#### Inspect content of a file and fetch RooDataSet from the file

In [ ]:
# Look at the content of the file using file.ls()
data = data_file.Get('dataReader.2016.Fit')
sig_data = sig_file.Get('sigMCReader.2016.Fit')

### Fit signal sample
#### Modelling

In [ ]:
Bmass = ROOT.RooRealVar("Bmass", "B Mass", 5.37, 5.2, 5.6)
mean  = ROOT.RooRealVar("mean", "Mean", 5.37, 5.3, 5.4)
width = ROOT.RooRealVar("width", "Width", 0.03, 0.001, 1)
sig_model = ROOT.RooGaussian("sig_model", "Signal Shape", Bmass, mean, width)

#### See how initial configuration of the model looks like

In [ ]:
sig_frame = Bmass.frame()
sig_data.plotOn(sig_frame)
sig_model.plotOn(sig_frame)
sig_frame.Draw()
ROOT.gPad.Draw()

#### Fit and again plot 

In [ ]:
result = sig_model.fitTo(sig_data, ROOT.RooFit.Save(), ROOT.RooFit.PrintLevel(0))

In [ ]:
sig_frame = Bmass.frame()
sig_data.plotOn(sig_frame)
sig_model.plotOn(sig_frame)
sig_frame.Draw()
ROOT.gPad.Draw()

#### Single Gaussian model did NOT work, try double Gaussian 

In [ ]:
# Defining double Guassian model with common mean
width1 = ROOT.RooRealVar("width1", "Width 1", 0.03, 0.001, 1)
width2 = ROOT.RooRealVar("width2", "Width 2", 0.034, 0.001, 1)
sig_model1 = ROOT.RooGaussian("sig_model1", "Signal Shape1", Bmass, mean, width1)
sig_model2 = ROOT.RooGaussian("sig_model2", "Signal Shape2", Bmass, mean, width2)
frac = ROOT.RooRealVar("frac", "frac", 0, 1)
pdfList = ROOT.RooArgList(sig_model1, sig_model2)
coefList = ROOT.RooArgList(frac)

dg_model = ROOT.RooAddPdf("dg_model", "Double Gausian Model", pdfList, coefList)

In [ ]:
dg_model.fitTo(sig_data)

In [ ]:
c1 = ROOT.TCanvas()
sig_frame = Bmass.frame()
sig_data.plotOn(sig_frame)
dg_model.plotOn(sig_frame)
sig_frame.Draw()
c1.Draw()

#### Double Guassian shape looks better though it is not perfect. See goodness of the fit by plotting residual/Pull distribution

## Residuals and Pull Plots

In [ ]:
res_sig_frame = sig_frame.emptyClone("frame_for_residue_of_signal")
pullHist = sig_frame.pullHist(sig_frame.getObject(0).GetName(), sig_frame.getObject(1).GetName())
res_sig_frame.addPlotable(pullHist, "P")
res_sig_frame.Draw()
c1.Draw()

#### You could try out different C++ modules in python. Here is the one to place residue plot below the original fit

In [ ]:
#Import C++ code. Can be executed only once unlike python import statements
ROOT.gInterpreter.ProcessLine('#include "../cpp/ResiduePlotter.cpp"')

In [ ]:
c1 = ROOT.TCanvas() # Declare Canvas
residue = ROOT.CustomRatioPlot('RooPlot')(sig_frame, res_sig_frame)
residue.Draw() # Draws all frames and TPads
c1.Draw()

#### Use shape of a signal from simulation to reduce number of free parameters

In [ ]:
# Fix parameters of signal PDF
width1.setConstant()
width2.setConstant()
mean.setConstant()

# Exponential function of a background 
slope = ROOT.RooRealVar('slope', 'Exp Slope', -5.1, -20, 1)
expo = ROOT.RooExponential("exp", "exp pdf title", Bmass, slope)

# Format final extended PDF
nSig = ROOT.RooRealVar('nSig', 'nSig', 230, 0, 300)
nBkgComb = ROOT.RooRealVar('nBkgComb', 'nBkgComb', 85, 0, 200)
f_finalM = ROOT.RooAddPdf("f_finalM", "Final Mass PDF", ROOT.RooArgList(expo, dg_model), ROOT.RooArgList(nBkgComb, nSig))

In [ ]:
# perform a fit and plot
fit_result = f_finalM.fitTo(data, ROOT.RooFit.Save())

c1 = ROOT.TCanvas()
frame = Bmass.frame()
data.plotOn(frame)
f_finalM.plotOn(frame)
f_finalM.plotOn(frame, ROOT.RooFit.Components('exp'), ROOT.RooFit.LineColor(2))
f_finalM.plotOn(frame, ROOT.RooFit.Components('dg_model'), ROOT.RooFit.LineColor(9))

frame2 = frame.emptyClone("frame_for_residue_of_final_pdf")
pullHist = frame.pullHist(frame.getObject(0).GetName(), frame.getObject(1).GetName())
frame2.addPlotable(pullHist, "P")
residue = ROOT.CustomRatioPlot('RooPlot')(frame, frame2)
residue.Draw()
c1.Draw()

### Save the PDFs for further use

In [ ]:
wspace = ROOT.RooWorkspace('wspace')
wspace.Import(f_finalM)
out_file = ROOT.TFile('data_fit_bmass.root', 'recreate')
out_file.WriteObject(wspace, 'wspace')
out_file.Close()

In [ ]:
wfile = ROOT.TFile('data_fit_bmass.root')
newspace = wfile.Get('wspace')
for i in newspace.allPdfs(): print(i)

### Toy Study

In [ ]:
width1.setConstant(True)
width2.setConstant(True)
mean.setConstant(False)
mcstudy = ROOT.RooMCStudy(
    f_finalM,
    ROOT.RooArgSet(Bmass),
    ROOT.RooFit.Binned(ROOT.kTRUE),
    ROOT.RooFit.Silence(),
    ROOT.RooFit.Extended(),
    ROOT.RooFit.FitOptions(
    ROOT.RooFit.Save(ROOT.kTRUE),
    ROOT.RooFit.PrintEvalErrors(0))
)

In [ ]:
mcstudy.generateAndFit(1000)

In [ ]:
# Make plots of the distributions of mean, error on mean and the pull of
# mean
frame1 = mcstudy.plotParam(mean, ROOT.RooFit.Bins(40))
frame2 = mcstudy.plotError(mean, ROOT.RooFit.Bins(40))
frame3 = mcstudy.plotPull(mean, ROOT.RooFit.Bins(40), ROOT.RooFit.FitGauss(ROOT.kTRUE))
 
# Plot distribution of minimized likelihood
frame4 = mcstudy.plotNLL(ROOT.RooFit.Bins(40))

In [ ]:
ROOT.gStyle.SetPalette(1)
ROOT.gStyle.SetOptStat(0)
c = ROOT.TCanvas("toystudy", "toystudy", 900, 600)
c.Divide(2, 2)
c.cd(1)
ROOT.gPad.SetLeftMargin(0.15)
frame1.GetYaxis().SetTitleOffset(1.4)
frame1.Draw()
c.cd(2)
ROOT.gPad.SetLeftMargin(0.15)
frame2.GetYaxis().SetTitleOffset(1.4)
frame2.Draw()
c.cd(3)
ROOT.gPad.SetLeftMargin(0.15)
frame3.GetYaxis().SetTitleOffset(1.4)
frame3.Draw()
c.cd(4)
ROOT.gPad.SetLeftMargin(0.15)
frame4.GetYaxis().SetTitleOffset(1.4)
frame4.Draw()
c.Draw()

### Toy study without RooMCStudy class

In [ ]:
# Retrieve some values from Original PDF
args = f_finalM.getParameters(data).Clone()
nSigVal = nSig.getVal()
nBkgCombVal = nBkgComb.getVal()
TotalEvents = nSigVal + nBkgCombVal
meanVal = mean.getVal()

Fluctuate from database value

In [ ]:
# Fluctuate and fit toy samples
meanArray = []
nllArray = []
for sample in range(100):
    gaus = ROOT.TF1("gaus", "exp(-0.5*x**2)", -.5, .5)
    significance = gaus.GetRandom()
    nSig.setVal(nSigVal + (significance*nSig.getError()) )
    nBkgComb.setVal(TotalEvents - nSig.getVal())
    mean.setVal(meanVal)

    sample_data = f_finalM.generate(Bmass, TotalEvents)
    sample_fit_result = f_finalM.fitTo(sample_data, ROOT.RooFit.Save(), ROOT.RooFit.PrintLevel(-3), ROOT.RooFit.Extended())
    if not (sample+1%30):
        print(f"Running sample {sample+1}")

    meanArray.append(mean.getVal())
    nllArray.append(sample_fit_result.minNll())
    

In [ ]:
# Plot Mean
c2 = ROOT.TCanvas()
toyMeanHist = ROOT.TH1D("meantoy", "mean in toy study", 40, 5.3 , 5.42)
for value in meanArray: toyMeanHist.Fill(value)
toyMeanHist.Draw()
ROOT.gPad.Draw()
c2.Draw()

#### Figure out what is wrong our strategy